In [7]:
import pandas as pd
import configparser

# Read the value from the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')
file_path = config.get('data', 'file_path')
# Read the data from the csv file
df = pd.read_csv(file_path, engine='pyarrow')

print(df.head())

    Symbol    Date   Open   High    Low  Close  Volume
0  VN30F1M  170810  757.9  757.9  745.5  745.9     367
1  VN30F1M  170811  746.0  746.4  743.4  745.7     510
2  VN30F1M  170814  745.9  748.9  745.7  748.9     707
3  VN30F1M  170815  750.3  750.9  748.0  748.0     946
4  VN30F1M  170816  747.7  748.4  745.1  748.1    1065


In [8]:
# Drop the rows with missing values
df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Symbol  1471 non-null   object 
 1   Date    1471 non-null   int64  
 2   Open    1471 non-null   float64
 3   High    1471 non-null   float64
 4   Low     1471 non-null   float64
 5   Close   1471 non-null   float64
 6   Volume  1471 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 80.6+ KB
None


In [36]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named "df"
# Convert the "Date" column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Map the day of the week to corresponding labels
day_labels = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
df['DoW'] = df['Date'].dt.dayofweek.map(lambda x: day_labels[x])

# Check if the date is the 3rd Thursday of the month and label it as "Th-expired"
DoWExp = config.get('derivatives', 'DoWExp')

df['DoW'] = np.where((df['Date'].dt.weekday == 3) & (df['Date'].dt.day <= 21) & (df['Date'].dt.day >= 15), DoWExp + '-exp', df['DoW'])

# Revert the "Date" column back to the original format
df['Date'] = df['Date'].dt.strftime('%y%m%d')

# Print the updated dataframe
print(df)


NoSectionError: No section: 'derivatives'

In [11]:
import numpy as np

# Create the FO column
df['FO'] = np.where(df['Open'] > df['Close'].shift(1), 'OU', 'OD')

# Create the FC column
df['FC'] = np.where(df['Close'] > df['Close'].shift(1), 'CU', 'CD')

# Calculate the percentage increase for FO
df['%O'] = np.where(df['FO'] == 'OU', (df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1) * 100, 0)

# Calculate the percentage decrease for FO
df['%O'] = np.where(df['FO'] == 'OD', (df['Close'].shift(1) - df['Open']) / df['Open'] * 100, df['%O'])

# Calculate the percentage increase for FC
df['%C'] = np.where(df['FC'] == 'CU', (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1) * 100, 0)

# Calculate the percentage decrease for FC
df['%C'] = np.where(df['FC'] == 'CD', (df['Close'].shift(1) - df['Close']) / df['Close'].shift(1) * 100, df['%C'])

# Print the updated dataframe
print(df)


       Symbol    Date    Open    High     Low   Close  Volume DoW  FO  FC   
0     VN30F1M  170810   757.9   757.9   745.5   745.9     367  Th  OD  CD  \
1     VN30F1M  170811   746.0   746.4   743.4   745.7     510  Fr  OU  CD   
2     VN30F1M  170814   745.9   748.9   745.7   748.9     707  Mo  OU  CU   
3     VN30F1M  170815   750.3   750.9   748.0   748.0     946  Tu  OU  CD   
4     VN30F1M  170816   747.7   748.4   745.1   748.1    1065  We  OD  CU   
...       ...     ...     ...     ...     ...     ...     ...  ..  ..  ..   
1466  VN30F1M  230622  1115.0  1121.0  1110.2  1114.0  178372  Th  OD  CD   
1467  VN30F1M  230623  1114.2  1123.5  1112.5  1119.1  214094  Fr  OU  CU   
1468  VN30F1M  230626  1118.0  1124.4  1111.7  1123.0  217308  Mo  OD  CU   
1469  VN30F1M  230627  1125.5  1125.9  1120.8  1125.2  132592  Tu  OU  CU   
1470  VN30F1M  230628  1127.0  1136.0  1126.5  1132.4  139703  We  OU  CU   

            %O        %C  
0          NaN       NaN  
1     0.013407  0.026

In [33]:
test_df = df[df['DoW'] == 'Th-exp']

# Count number of CD and CU in test_df
print(test_df['FC'].value_counts())

# calculate the average %C for CD and CU
print(test_df.groupby('FC')['%C'].mean())

# Count number of OU and OD in test_df
print(test_df['FO'].value_counts())

# calculate the average %O for OD and OU
print(test_df.groupby('FO')['%O'].mean())

FC
CU    36
CD    34
Name: count, dtype: int64
FC
CD    0.907139
CU    1.120489
Name: %C, dtype: float64
FO
OD    36
OU    34
Name: count, dtype: int64
FO
OD    0.348921
OU    0.210588
Name: %O, dtype: float64


In [16]:
# Count number of CD and CU in df, excluding Th-exp
df_excl_th = df[df['DoW'] != 'Th-exp']
print(df_excl_th['FC'].value_counts())

# calculate the average %C for CD and CU
print(df_excl_th.groupby('FC')['%C'].mean())

FC
CU    764
CD    637
Name: count, dtype: int64
FC
CD    1.127773
CU    1.005427
Name: %C, dtype: float64


In [17]:
df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,FO,FC,%O,%C
0,VN30F1M,170810,757.9,757.9,745.5,745.9,367,Th,OD,CD,NaN,NaN
1,VN30F1M,170811,746.0,746.4,743.4,745.7,510,Fr,OU,CD,0.013407,0.026813
2,VN30F1M,170814,745.9,748.9,745.7,748.9,707,Mo,OU,CU,0.026820,0.429127
3,VN30F1M,170815,750.3,750.9,748.0,748.0,946,Tu,OU,CD,0.186941,0.120176
4,VN30F1M,170816,747.7,748.4,745.1,748.1,1065,We,OD,CU,0.040123,0.013369
...,...,...,...,...,...,...,...,...,...,...,...,...
1466,VN30F1M,230622,1115.0,1121.0,1110.2,1114.0,178372,Th,OD,CD,0.044843,0.134469
1467,VN30F1M,230623,1114.2,1123.5,1112.5,1119.1,214094,Fr,OU,CU,0.017953,0.457810
1468,VN30F1M,230626,1118.0,1124.4,1111.7,1123.0,217308,Mo,OD,CU,0.098390,0.348494
1469,VN30F1M,230627,1125.5,1125.9,1120.8,1125.2,132592,Tu,OU,CU,0.222618,0.195904


In [34]:
import pandas as pd

# Assuming your dataframe is named "df"

# Define the categories
categories = ['CU', 'CD']

# Create a pivot table to calculate the count
pivot_df = pd.pivot_table(df, index='DoW', values='FO', columns='FC', aggfunc='count', fill_value=0)

# Calculate the total count
pivot_df['Total'] = pivot_df.sum(axis=1)

# Calculate the percentages
for category in categories:
    pivot_df[f'%{category}'] = pivot_df[category] / pivot_df['Total']

# Reorder the columns
columns = categories + [f'%{category}' for category in categories] + ['Total']
pivot_df = pivot_df.reindex(columns=columns)

# Reindex the rows to match the desired order
day_labels = ['Su', 'Mo', 'Tu', 'We', 'Th','Th-exp', 'Fr', 'Sa']
pivot_df = pivot_df.reindex(day_labels)

# Drop Total column
pivot_df.drop('Total', axis=1, inplace=True)

pivot_df.dropna(inplace=True)

pivot_df

FC,CU,CD,%CU,%CD
DoW,,,,
Mo,147.0,140.0,0.512195,0.487805
Tu,169.0,127.0,0.570946,0.429054
We,171.0,124.0,0.579661,0.420339
Th,115.0,112.0,0.506608,0.493392
Th-exp,36.0,34.0,0.514286,0.485714
Fr,162.0,134.0,0.547297,0.452703


In [35]:
import pandas as pd

# Assuming your dataframe is named "df"

# Define the categories
categories = ['OU', 'OD']

# Create a pivot table to calculate the count
pivot_df = pd.pivot_table(df, index='DoW', values='FC', columns='FO', aggfunc='count', fill_value=0)

# Calculate the total count
pivot_df['Total'] = pivot_df.sum(axis=1)

# Calculate the percentages
for category in categories:
    pivot_df[f'%{category}'] = pivot_df[category] / pivot_df['Total']

# Reorder the columns
columns = categories + [f'%{category}' for category in categories] + ['Total']
pivot_df = pivot_df.reindex(columns=columns)

# Reindex the rows to match the desired order
day_labels = ['Su', 'Mo', 'Tu', 'We', 'Th','Th-exp', 'Fr', 'Sa']
pivot_df = pivot_df.reindex(day_labels)

# Drop Total column
pivot_df.drop('Total', axis=1, inplace=True)

# Drop the rows with missing values
pivot_df.dropna(inplace=True)

pivot_df

FO,OU,OD,%OU,%OD
DoW,,,,
Mo,174.0,113.0,0.606272,0.393728
Tu,163.0,133.0,0.550676,0.449324
We,166.0,129.0,0.562712,0.437288
Th,129.0,98.0,0.568282,0.431718
Th-exp,34.0,36.0,0.485714,0.514286
Fr,168.0,128.0,0.567568,0.432432
